In [1]:
import numpy as np

In [17]:
# Data Load step
data = np.load('/content/new_forecasts.npy', allow_pickle=True)
stocks_data = []
for k, v in data.item().items():
  
  stocks_data.append(v)

stocks_data = np.array(stocks_data)
stocks_data[0][0]

72.8499984741211

In [18]:
# Single graph node
class Node:
  def __init__(self,val,pred,ID,path):
    self.val = val
    self.pred = pred
    self.ID = ID
    self.path = path


In [19]:
class Strat_predictor:
  def __init__(self,arr):
    self.data = arr
  
  def create_row(self,arr):
    n, = np.shape(arr)
    k = n*(n-1)//2
    
    grid = np.zeros((k,1))
    count = 0
    for i in range(n):
        for j in range(i+1,n):
            grid[count] = ((arr[j] - arr[i])/arr[i])
            count+=1
    grid = grid.reshape(1,k)
    return grid

  def create_grid(self,arr):
    m,n = np.shape(arr)
    k = n*(n-1)//2
    delta = []
    for i in range(m):
        delt = self.create_row(arr[i])
        delta.append(delt)
    
    delta2 = np.array((delta)).reshape(m,k)
    return delta2,n
  
  def create_matrix(self):
    self.table,num_nodes = self.create_grid(self.data)
    self.t = np.array(np.max(self.table,axis = 0))

    # self.adj_graph = np.full((num_nodes,num_nodes),None)
    
    # counter = 0
    # for i in range(num_nodes):
    #   for j in range(num_nodes):
    #     if i==j:
    #       self.adj_graph[i][j] = 0
    #     elif j > i:
    #       continue
    #     else:
    #       self.adj_graph[i][j] = self.t[counter]
    #       counter+=1
    self.adj_graph = np.full((num_nodes,num_nodes),None)
    counter = num_nodes-1

    start = 0
    for i in range(num_nodes-1):
      self.adj_graph[i][i+1:] = self.t[start:start + counter]
      start = start+counter
      counter-=1

    for i in range(num_nodes):
      self.adj_graph[i][i] = 0

    self.adj_graph = self.adj_graph.T
    #print(self.adj_graph)
    return num_nodes
  
  def setup_graph(self):
    self.num_nodes = self.create_matrix()
    self.nodes = []
    for i in range(self.num_nodes):
      self.nodes.append(Node(0,list(range(i)),i,[]))
  
  def get_strat(self):
    self.setup_graph()
    gstack = []
    stack = []
    flag = (0,0)
    for i,j in enumerate(self.nodes):
      row = self.adj_graph[i]
      for p in j.pred:
        value = self.nodes[p].val + row[p]
        if value > j.val:
          flag = (p,i)

        j.val = max(j.val,value)
      #print(flag)
      if j == 0:
        j.path = [0]
      else:
        s,e = flag
        j.path = self.nodes[s].path + [e]
      gstack.append((flag))

  def check(self):
    self.get_strat()
    best_node_val = 0
    for k in range(self.num_nodes):
      #print(self.nodes[k].val,self.nodes[k].path)
      vals = self.nodes[k].val
      if vals > best_node_val:
        best_node_index = k
      best_node_val = max(best_node_val,vals)
    
    best_node_path = self.nodes[best_node_index].path
    p_path = best_node_path

    path = list(zip(p_path[:-1],p_path[1:]))
    return path
    
  def predict(self,path):
    self.strat = []
    for buy_day,sell_day in path:
      stock_val = self.adj_graph[sell_day][buy_day]
      stock,delta = np.where(self.table == stock_val)
      self.strat.append(((stock),(buy_day,sell_day)))
      #print(self.strat)
    return self.strat

  def display(self):
    money = 1000
    for stock,(b,s) in self.strat:
      #print(stock)
      #index = int(stock)
      index = stock[0]
      #print(index)
      print('Buy Stock with ID',index,'at end of day',b,'and sell at end of day',s)
      stock_num = money/self.data[index][b]
      money = stock_num*self.data[index][s]
    print("Profit made on investement of 1000 Dollars is",money-1000)

In [20]:
#strat

In [28]:
maxminval = []
for i in range(stocks_data.shape[0]):
  f = (np.max(stocks_data[i]) - np.min(stocks_data[i]))/np.min(stocks_data[i])
  mxd,mnd = np.argmax(stocks_data[i]),np.argmin(stocks_data[i])
  if mnd > mxd:
    continue
  maxminval.append((f,mxd,mnd,i))

print('Increasing stocks number is',len(maxminval))
maxminval.sort(key = lambda x:-x[0])

print('The best performing stock is',maxminval[0][3],'Naive max Value',maxminval[0][0])


Increasing stocks number is 30
The best performing stock is 6 Naive max Value 0.041654176324653576


In [29]:
# Naive Estimator
(stocks_data[6][5] - stocks_data[6][0])/stocks_data[6][0]

0.041654176324653576

In [30]:
# Our Graph based predictor
h = Strat_predictor(stocks_data)
y = h.check()
strat = h.predict(y)
h.display()

Buy Stock with ID 6 at end of day 0 and sell at end of day 1
Buy Stock with ID 92 at end of day 1 and sell at end of day 2
Buy Stock with ID 17 at end of day 2 and sell at end of day 3
Buy Stock with ID 92 at end of day 3 and sell at end of day 4
Buy Stock with ID 6 at end of day 4 and sell at end of day 5
Profit made on investement of 1000 Dollars is 69.62530486915284
